### <div style="color:brown;text-align:center"> Researcher: Akinsulure Akintunde </div>

### <div style="color:red">Problem Statement</div>

The objective is to extract meaningful topics using two different topic modelling approaches: LDA and BERTopic. The task is to identify thematic structures within the movie synopses (using the synopsis column in the available data) and compare the topics generated by the traditional method (LDA) with those produced by the more recent, embedding-based method (BERTopic).

---